# New model

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, 
                                   Dropout, BatchNormalization, GlobalAveragePooling2D)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau, ModelCheckpoint)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

## Load and preprocessing data

In [16]:
def load_and_preprocess_data():
    """Load data with improved preprocessing"""
    try:
        X_train = np.loadtxt('input.csv', delimiter=',')
        X_test = np.loadtxt('input_test.csv', delimiter=',')
        Y_train = np.loadtxt('labels.csv', delimiter=',')
        Y_test = np.loadtxt('labels_test.csv', delimiter=',')
    except ValueError:
        import pandas as pd
        X_train = pd.read_csv('input.csv').to_numpy()
        X_test = pd.read_csv('input_test.csv').to_numpy()
        Y_train = pd.read_csv('labels.csv').to_numpy()
        Y_test = pd.read_csv('labels_test.csv').to_numpy()
    
    # Reshape data
    X_train = X_train.reshape(-1, 100, 100, 3)
    X_test = X_test.reshape(-1, 100, 100, 3)
    Y_train = Y_train.reshape(-1, 1)
    Y_test = Y_test.reshape(-1, 1)
    
    # Normalize pixel values
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    
    # Per-channel normalization 
    # Compute mean and std per channel on training data
    train_mean = np.mean(X_train, axis=(0, 1, 2))
    train_std = np.std(X_train, axis=(0, 1, 2))
    
    # Apply normalization
    X_train = (X_train - train_mean) / (train_std + 1e-7)
    X_test = (X_test - train_mean) / (train_std + 1e-7)
    
    print(f"Training data shape: {X_train.shape}")
    print(f"Test data shape: {X_test.shape}")
    print(f"Training labels distribution: {np.bincount(Y_train.astype(int).flatten())}")
    
    return X_train, X_test, Y_train, Y_test

# data augmentatio

In [17]:
def create_data_generators(X_train, Y_train, X_test, Y_test):
    """Create data generators with more aggressive augmentation"""
    train_datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        zoom_range=0.3,
        shear_range=0.2,
        brightness_range=[0.8, 1.2],
        fill_mode='nearest'
    )
    
    # No augmentation for validation data
    val_datagen = ImageDataGenerator()
    
    train_generator = train_datagen.flow(X_train, Y_train, batch_size=32)
    val_generator = val_datagen.flow(X_test, Y_test, batch_size=32, shuffle=False)
    
    return train_generator, val_generator


# CNN architecture

In [18]:
def create_improved_model():
    """Create an improved CNN model with better architecture"""
    model = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Third Convolutional Block
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Fourth Convolutional Block
        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        Dropout(0.25),
        
        # Global Average Pooling instead of Flatten
        GlobalAveragePooling2D(),
        
        # Dense layers
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    return model


# Transfer learning model (VGG16-based)

In [19]:
def create_transfer_learning_model():
    """Create a transfer learning model using VGG16"""
    from tensorflow.keras.applications import VGG16
    from tensorflow.keras.models import Model
    
    # Load pre-trained VGG16 model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
    
    # Freeze early layers
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    
    # Add custom classifier
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Training 

In [20]:
def train_model(model, train_generator, val_generator, model_name="enhanced_model"):
    """Train the model with advanced callbacks"""
    
    # Compile with optimized parameters
    optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Advanced callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            f'{model_name}_best.h5',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]
    
    # Train the model
    history = model.fit(
        train_generator,
        epochs=100,  # More epochs with early stopping
        validation_data=val_generator,
        callbacks=callbacks,
        verbose=1
    )
    
    return history


# Evaluation and visualization

In [21]:
def evaluate_and_visualize(model, X_test, Y_test):
    """Comprehensive model evaluation"""
    
    # Make predictions
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int)
    
    # Calculate accuracy
    accuracy = np.mean(y_pred.flatten() == Y_test.flatten())
    print(f"Final Test Accuracy: {accuracy:.4f}")
    
    # Confusion Matrix
    cm = confusion_matrix(Y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Dog', 'Cat'], yticklabels=['Dog', 'Cat'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix - Enhanced Model')
    plt.show()
    
    # Classification Report
    print("\nClassification Report:")
    print(classification_report(Y_test, y_pred, target_names=['Dog', 'Cat']))
    
    return accuracy


# Ensemble method

In [22]:
def create_ensemble_prediction(models, X_test):
    """Create ensemble predictions from multiple models"""
    predictions = []
    
    for model in models:
        pred = model.predict(X_test)
        predictions.append(pred)
    
    # Average predictions
    ensemble_pred = np.mean(predictions, axis=0)
    return (ensemble_pred > 0.5).astype(int)

# Main execution function
def main():
    """Main function to run the entire pipeline"""
    
    print("="*50)
    print("ENHANCED DOG VS CAT CLASSIFICATION")
    print("="*50)
    
    # Step 1: Load and preprocess data
    print("\n1. Loading and preprocessing data...")
    X_train, X_test, Y_train, Y_test = load_and_preprocess_data()
    
    # Step 2: Create data generators
    print("\n2. Creating data generators...")
    train_gen, val_gen = create_data_generators(X_train, Y_train, X_test, Y_test)
    
    # Step 3: Train improved CNN model
    print("\n3. Training improved CNN model...")
    improved_model = create_improved_model()
    print(f"Model parameters: {improved_model.count_params():,}")
    
    history1 = train_model(improved_model, train_gen, val_gen, "improved_cnn")
    acc1 = evaluate_and_visualize(improved_model, X_test, Y_test)
    
    # Step 4: Train transfer learning model
    print("\n4. Training transfer learning model...")
    transfer_model = create_transfer_learning_model()
    print(f"Transfer model parameters: {transfer_model.count_params():,}")
    
    # Recreate generators for transfer learning
    train_gen2, val_gen2 = create_data_generators(X_train, Y_train, X_test, Y_test)
    history2 = train_model(transfer_model, train_gen2, val_gen2, "transfer_learning")
    acc2 = evaluate_and_visualize(transfer_model, X_test, Y_test)
    

# prediction

In [23]:
def create_ensemble_prediction(models, X_test):
    """Create ensemble predictions from multiple models"""
    predictions = []
    
    for model in models:
        pred = model.predict(X_test)
        predictions.append(pred)
    
    # Average predictions
    ensemble_pred = np.mean(predictions, axis=0)
    return (ensemble_pred > 0.5).astype(int)

# Main execution function
def main():
    """Main function to run the entire pipeline"""
    
    print("="*50)
    print("ENHANCED DOG VS CAT CLASSIFICATION")
    print("="*50)
    
    # Step 1: Load and preprocess data
    print("\n1. Loading and preprocessing data...")
    X_train, X_test, Y_train, Y_test = load_and_preprocess_data()
    
    # Step 2: Create data generators
    print("\n2. Creating data generators...")
    train_gen, val_gen = create_data_generators(X_train, Y_train, X_test, Y_test)
    
    # Step 3: Train improved CNN model
    print("\n3. Training improved CNN model...")
    improved_model = create_improved_model()
    print(f"Model parameters: {improved_model.count_params():,}")
    
    history1 = train_model(improved_model, train_gen, val_gen, "improved_cnn")
    acc1 = evaluate_and_visualize(improved_model, X_test, Y_test)
    
    # Step 4: Train transfer learning model
    print("\n4. Training transfer learning model...")
    transfer_model = create_transfer_learning_model()
    print(f"Transfer model parameters: {transfer_model.count_params():,}")
    
    # Recreate generators for transfer learning
    train_gen2, val_gen2 = create_data_generators(X_train, Y_train, X_test, Y_test)
    history2 = train_model(transfer_model, train_gen2, val_gen2, "transfer_learning")
    acc2 = evaluate_and_visualize(transfer_model, X_test, Y_test)
    
    # Step 5: Ensemble prediction
    print("\n5. Creating ensemble prediction...")
    ensemble_pred = create_ensemble_prediction([improved_model, transfer_model], X_test)
    ensemble_acc = np.mean(ensemble_pred.flatten() == Y_test.flatten())
    print(f"Ensemble Accuracy: {ensemble_acc:.4f}")
    
    # Final comparison
    print("\n" + "="*50)
    print("FINAL RESULTS COMPARISON:")
    print("="*50)
    print(f"Improved CNN Accuracy: {acc1:.4f}")
    print(f"Transfer Learning Accuracy: {acc2:.4f}")
    print(f"Ensemble Accuracy: {ensemble_acc:.4f}")
    print("="*50)
    
    return improved_model, transfer_model, ensemble_acc

# Additional utility functions
def plot_training_history(history):
    """Plot training history"""
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def show_prediction_examples(model, X_test, Y_test, num_examples=8):
    """Show prediction examples"""
    indices = np.random.choice(len(X_test), num_examples, replace=False)
    
    plt.figure(figsize=(16, 8))
    for i, idx in enumerate(indices):
        plt.subplot(2, 4, i+1)
        
        # Denormalize image for display (if normalized)
        img = X_test[idx]
        if img.min() < 0:  # If normalized
            img = (img - img.min()) / (img.max() - img.min())
        
        plt.imshow(img)
        
        pred_prob = model.predict(np.expand_dims(X_test[idx], axis=0))[0][0]
        pred_class = "Cat" if pred_prob > 0.5 else "Dog"
        true_class = "Cat" if Y_test[idx][0] > 0.5 else "Dog"
        
        color = 'green' if pred_class == true_class else 'red'
        plt.title(f'True: {true_class}\nPred: {pred_class} ({pred_prob:.2f})', 
                 color=color)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Main

In [24]:
if __name__ == "__main__":
    improved_model, transfer_model, final_accuracy = main()

ENHANCED DOG VS CAT CLASSIFICATION

1. Loading and preprocessing data...
Training data shape: (2000, 100, 100, 3)
Test data shape: (400, 100, 100, 3)
Training labels distribution: [1000 1000]

2. Creating data generators...

3. Training improved CNN model...
Model parameters: 849,313


d:\Code\DogVsCat-classification-using-CNN\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\Code\DogVsCat-classification-using-CNN\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - accuracy: 0.5031 - loss: 1.1193
Epoch 1: val_accuracy improved from None to 0.50000, saving model to improved_cnn_best.h5


63/63 ━━━━━━━━━━━━━━━━━━━━ 21s 247ms/step - accuracy: 0.5215 - loss: 1.0778 - val_accuracy: 0.5000 - val_loss: 0.8777 - learning_rate: 0.0010
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - accuracy: 0.5401 - loss: 1.0289
Epoch 2: val_accuracy improved from 0.50000 to 0.51250, saving model to improved_cnn_best.h5


63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 244ms/step - accuracy: 0.5295 - loss: 1.0321 - val_accuracy: 0.5125 - val_loss: 0.7163 - learning_rate: 0.0010
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - accuracy: 0.5320 - loss: 0.9189
Epoch 3: val_accuracy did not improve from 0.51250
63/63 ━━━━━━━━━━━━━━━━━━━━ 15s 230ms/step - accuracy: 0.5485 - loss: 0.9241 - val_accuracy: 0.5050 - val_loss: 0.7891 - learning_rate: 0.0010
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step - accuracy: 0.5471 - loss: 0.8873
Epoch 4: val_accuracy improved from 0.51250 to 0.53250, saving model to improved_cnn_best.h5


63/63 ━━━━━━━━━━━━━━━━━━━━ 15s 242ms/step - accuracy: 0.5375 - loss: 0.8883 - val_accuracy: 0.5325 - val_loss: 0.6815 - learning_rate: 0.0010
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.5804 - loss: 0.7711
Epoch 5: val_accuracy improved from 0.53250 to 0.58500, saving model to improved_cnn_best.h5


63/63 ━━━━━━━━━━━━━━━━━━━━ 15s 236ms/step - accuracy: 0.5580 - loss: 0.8012 - val_accuracy: 0.5850 - val_loss: 0.7049 - learning_rate: 0.0010
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step - accuracy: 0.5598 - loss: 0.7739
Epoch 6: val_accuracy did not improve from 0.58500
63/63 ━━━━━━━━━━━━━━━━━━━━ 15s 240ms/step - accuracy: 0.5680 - loss: 0.7673 - val_accuracy: 0.5800 - val_loss: 0.6615 - learning_rate: 0.0010
Epoch 7/100
26/63 ━━━━━━━━━━━━━━━━━━━━ 8s 224ms/step - accuracy: 0.6032 - loss: 0.7160

KeyboardInterrupt: 

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [26]:
def show_prediction_examples(model, X_test, Y_test, model_name="Model", num_examples=8):
    """Show prediction examples with images, true labels, and predicted labels"""
    
    # Select random examples
    indices = np.random.choice(len(X_test), num_examples, replace=False)
    
    # Create subplot
    plt.figure(figsize=(20, 10))
    
    for i, idx in enumerate(indices):
        plt.subplot(2, 4, i+1)
        
        # Get the image
        img = X_test[idx].copy()
        
        # Denormalize image for display (since we normalized it during preprocessing)
        # Reverse the per-channel normalization if you applied it
        if img.min() < 0:  # If the image was normalized with mean/std
            img = (img - img.min()) / (img.max() - img.min())
        else:
            img = np.clip(img, 0, 1)  # Ensure values are in [0,1] range
        
        # Display image
        plt.imshow(img)
        
        # Make prediction
        pred_prob = model.predict(np.expand_dims(X_test[idx], axis=0), verbose=0)[0][0]
        pred_class = "Cat" if pred_prob > 0.5 else "Dog"
        true_class = "Cat" if Y_test[idx][0] > 0.5 else "Dog"
        
        # Set title color based on correctness
        color = 'green' if pred_class == true_class else 'red'
        
        # Create title with prediction confidence
        title = f'True: {true_class}\nPred: {pred_class}\nConf: {pred_prob:.3f}'
        plt.title(title, color=color, fontsize=12, fontweight='bold')
        
        # Remove axis
        plt.axis('off')
    
    plt.suptitle(f'{model_name} - Prediction Examples\n(Green=Correct, Red=Incorrect)', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

def show_single_prediction(model, X_test, Y_test, model_name="Model"):
    """Show a single random prediction - similar to your original code"""
    
    # Pick a random image
    idx = random.randint(0, len(Y_test) - 1)
    
    # Display the image
    img = X_test[idx].copy()
    if img.min() < 0:  # Denormalize if needed
        img = (img - img.min()) / (img.max() - img.min())
    
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    
    # Get true label
    true_label = "Cat" if Y_test[idx, 0] > 0.5 else "Dog"
    
    # Make prediction
    pred_prob = model.predict(np.expand_dims(X_test[idx], axis=0), verbose=0)
    pred_class = "Cat" if pred_prob > 0.5 else "Dog"
    
    # Set title
    plt.title(f"True label: {true_label}", fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.show()
    
    # Print prediction
    print(f"Prediction: {pred_class}")
    print(f"True label: {true_label}")
    print(f"Confidence: {pred_prob[0][0]:.4f}")
    if pred_class == true_label:
        print("✅ CORRECT PREDICTION!")
    else:
        print("❌ INCORRECT PREDICTION!")

def compare_model_predictions(models, model_names, X_test, Y_test, num_examples=4):
    """Compare predictions from multiple models side by side"""
    
    indices = np.random.choice(len(X_test), num_examples, replace=False)
    
    fig, axes = plt.subplots(num_examples, len(models) + 1, figsize=(20, 5 * num_examples))
    
    for i, idx in enumerate(indices):
        # Show original image
        img = X_test[idx].copy()
        if img.min() < 0:
            img = (img - img.min()) / (img.max() - img.min())
        
        axes[i, 0].imshow(img)
        true_class = "Cat" if Y_test[idx][0] > 0.5 else "Dog"
        axes[i, 0].set_title(f'True: {true_class}', fontweight='bold')
        axes[i, 0].axis('off')
        
        # Show predictions from each model
        for j, (model, name) in enumerate(zip(models, model_names)):
            pred_prob = model.predict(np.expand_dims(X_test[idx], axis=0), verbose=0)[0][0]
            pred_class = "Cat" if pred_prob > 0.5 else "Dog"
            
            axes[i, j + 1].imshow(img)
            color = 'green' if pred_class == true_class else 'red'
            axes[i, j + 1].set_title(f'{name}\nPred: {pred_class}\nConf: {pred_prob:.3f}', 
                                   color=color, fontweight='bold')
            axes[i, j + 1].axis('off')
    
    plt.tight_layout()
    plt.show()

In [27]:
print("\n" + "="*60)
print("VISUAL PREDICTION EXAMPLES")
print("="*60)

# Show examples from your best model (transfer learning model)
show_prediction_examples(transfer_model, X_test, Y_test, "Transfer Learning Model", num_examples=8)

# Show a single prediction (like your original code)
print("\nSingle Random Prediction:")
show_single_prediction(transfer_model, X_test, Y_test, "Transfer Learning Model")

# If you have both models, compare their predictions
if 'improved_model' in locals() and 'transfer_model' in locals():
    print("\nModel Comparison:")
    compare_model_predictions([improved_model, transfer_model], 
                            ["Enhanced CNN", "Transfer Learning"], 
                            X_test, Y_test, num_examples=4)

# =============================================================================
# ADDITIONAL: Show best and worst predictions
# =============================================================================

def show_best_worst_predictions(model, X_test, Y_test, model_name="Model"):
    """Show the most confident correct and incorrect predictions"""
    
    # Get all predictions
    all_preds = model.predict(X_test, verbose=0)
    
    # Calculate confidence for each prediction
    confidences = np.abs(all_preds.flatten() - 0.5)  # Distance from 0.5 (uncertainty)
    
    # Get correct and incorrect predictions
    pred_classes = (all_preds.flatten() > 0.5).astype(int)
    true_classes = Y_test.flatten().astype(int)
    correct_mask = pred_classes == true_classes
    
    # Find most confident correct predictions
    correct_indices = np.where(correct_mask)[0]
    correct_confidences = confidences[correct_indices]
    most_confident_correct = correct_indices[np.argsort(correct_confidences)[-4:]]
    
    # Find most confident incorrect predictions
    incorrect_indices = np.where(~correct_mask)[0]
    if len(incorrect_indices) > 0:
        incorrect_confidences = confidences[incorrect_indices]
        most_confident_incorrect = incorrect_indices[np.argsort(incorrect_confidences)[-4:]]
    else:
        most_confident_incorrect = []
    
    # Display results
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    # Show most confident correct predictions
    for i, idx in enumerate(most_confident_correct):
        img = X_test[idx].copy()
        if img.min() < 0:
            img = (img - img.min()) / (img.max() - img.min())
        
        axes[0, i].imshow(img)
        true_class = "Cat" if Y_test[idx][0] > 0.5 else "Dog"
        pred_prob = all_preds[idx][0]
        pred_class = "Cat" if pred_prob > 0.5 else "Dog"
        
        axes[0, i].set_title(f'✅ CORRECT\nTrue: {true_class}\nConf: {pred_prob:.3f}', 
                           color='green', fontweight='bold')
        axes[0, i].axis('off')
    
    # Show most confident incorrect predictions
    for i in range(4):
        if i < len(most_confident_incorrect):
            idx = most_confident_incorrect[i]
            img = X_test[idx].copy()
            if img.min() < 0:
                img = (img - img.min()) / (img.max() - img.min())
            
            axes[1, i].imshow(img)
            true_class = "Cat" if Y_test[idx][0] > 0.5 else "Dog"
            pred_prob = all_preds[idx][0]
            pred_class = "Cat" if pred_prob > 0.5 else "Dog"
            
            axes[1, i].set_title(f'❌ INCORRECT\nTrue: {true_class}\nPred: {pred_class}\nConf: {pred_prob:.3f}', 
                               color='red', fontweight='bold')
            axes[1, i].axis('off')
        else:
            axes[1, i].axis('off')
    
    plt.suptitle(f'{model_name} - Most Confident Predictions', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Show best and worst predictions
print("\nMost Confident Predictions:")
show_best_worst_predictions(transfer_model, X_test, Y_test, "Transfer Learning Model")

print("\n🎉 Visual prediction analysis completed!")


VISUAL PREDICTION EXAMPLES


NameError: name 'X_test' is not defined